In [1]:


# note to prompt:
#      Tagging other users is a positive engagement!

# when doing insight analysis: remove bot messages, remove PR messages, remove messages that is likely sent by the seller themselves


# Which product or features of the product created more positive engagement?
#      (the direct value for this one is a bit less clear since we don't know which exact product they are engaging with)
#       (but at least we can use timestamp as an indicator of promotional events)

# Is there seasonality in promotional events?

#      If so, what are the peak periods for engagement?

#      Are there specific products that perform better during certain times of the week/day/month?

# How does engagement curve/population density/ look over time?

# Which descriptions created more positive engagements?
# Which descriptions attracted more feedback?



# Instructions for running and extending your work


# # build a chat functionality

# """
# I would like to have 'product identification' as an input to help track product's performance over time well'
# If this can't be generated, I would create a object recognition pipeline to extract these through scraping instagram.
# """
# # 

In [1]:
import sys
import os
src_path = os.path.abspath(os.pardir)
sys.path.append(src_path)


from src.data_prep import load_comments, dedup_near, remove_noise, enrich
INPUT_CSV = "../data/engagements.csv"
FIGDIR = "../outputs/figures"
MIN_COMMENTS_PER_POST = 20  # avoid tiny denominators when ranking posts

os.makedirs(FIGDIR, exist_ok=True)
import pandas as pd

import pandas as pd
df = pd.read_csv(INPUT_CSV)
original_len = len(df)


df = load_comments(INPUT_CSV)
df = dedup_near(df)
df = remove_noise(df)
df = enrich(df)

print(f"removed {original_len - len(df)} rows")

/Users/muduo/GitHub/codes/untitled folder/try/src/data_prep.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | txt.str.contains(r"([a-z])\1{4,}", regex=True, na=False)  # e.g., heyyyyy


removed 7666 rows


In [2]:
import pandas as pd

from src.metrics import (
    per_post_metrics,
    seasonality_tables,
    detect_promo_spikes,
    caption_feature_correlations,
    caption_keywords,
    assign_caption_terms_to_posts,
    term_level_engagement,
)


posts = per_post_metrics(df)
# Seasonality
vol, tag = seasonality_tables(df)
daily = detect_promo_spikes(df)

# Descriptions → feedback (correlations)
cor = caption_feature_correlations(posts)
cor.to_csv(FIGDIR + "/caption_feature_correlations.csv", index=False)

# Product/feature proxies from captions
top_terms = caption_keywords(df, min_df=15, top_k=40)
posts_terms = assign_caption_terms_to_posts(posts, top_terms)
term_df = term_level_engagement(posts_terms, top_terms)


Below is the main plot I've managed to complete:

1. The curve is plotted for all engagement posts
2. Stats is retrieved for top posts
3. Descriptions is provided

In [3]:
from src.viz import (
    plot_post_engagement_trajectories,
    plot_seasonality_heatmaps,
    plot_top_posts,
    plot_term_leaders
)

# After you have df (cleaned comments) and posts = per_post_metrics(df)
fig, axes, path, table = plot_post_engagement_trajectories(
    df_comments=df,
    posts=posts,
    out_path="../outputs/figures/post_trajectories.png",
    top_k=5,
    select_by="eng_score",   # or "comments"
    freq="H",                # "D" for daily
)
# print(table.head())  # P1..Pk with stats + T50 + caption snippet




/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq)
/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq)
/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq)
/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq)
/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(freq)
/Users/muduo/GitHub/codes/untitled folder/try/src/viz.py:94: FutureWarning: 'H' is deprecated and will be removed i

,timestamp,media_id,media_caption,comment_text
0,2025-03-01 00:13:57.153000+00:00,1090986906404998,"Soft skin, soft life 🩷🌸🫧 get your hands on thi...",I bet this is good
1,2025-03-01 00:23:06.879000+00:00,17950254656929862,Why use one scrub when you can use them all at...,i know this smells so good
2,2025-03-01 00:04:05.094000+00:00,1090109319826090,Morning routine with Tree Hut 🍊🫧 Now available...,Love it
3,2025-03-01 00:41:59.467000+00:00,1098364052333950,Why use one scrub when you can use them all at...,Please carry these in Canada! I miss them so m...
4,2025-03-01 02:21:29.715000+00:00,1083943630442659,Vanilla Serum-Infused Hand Wash: A sweet escap...,I love it ..✌️
...,...,...,...,...
17836,2025-04-02 18:27:20.065000+00:00,1125861369584218,that's that me espresso,Anthony Baggio I want
17837,2025-04-02 18:28:41.488000+00:00,18076572166794234,Let us know what your Spring Vibe is going to ...,A1!😍
17838,2025-04-02 18:31:56.153000+00:00,1125861369584218,that's that me espresso,Cydney Shue for you babes
17839,2025-04-02 18:29:08.782000+00:00,1125861369584218,that's that me espresso,Bryanna Crable omg I need 😂😂


In [ ]:
# some other partial ideas and potentially meaningful datapoints not fully flushed out yet
plot_seasonality_heatmaps(
    vol, tag,
    out_volume_path="../outputs/figures/seasonality_volume.png",
    out_tag_rate_path="../outputs/figures/seasonality_tag_rate.png",
)
plot_top_posts(posts, out_csv_path="../outputs/figures/top_posts_by_eng_score.csv", top_n=20)
plot_term_leaders(term_df, out_csv_path="../outputs/figures/top_terms_by_eng.csv", top_n=20)


# Quick, human-readable takeaways to drop in your report
print("\n=== Top posts by engagement score ===")
print(posts.sort_values("eng_score", ascending=False).head(10)[
    ["media_id","comments","tag_rate","pos_rate","eng_score","first_ts"]
])

print("\n=== Caption feature correlations (higher is stronger) ===")
print(cor.groupby("metric").head(3))


print("\n=== Top caption terms by avg_eng_score (proxy for product/features) ===")
print(term_df.head(10))


print("\n=== Promo spikes (z>=2) ===")
print(daily[daily["is_spike"]][["n","z"]].tail(10))
